In [1]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pymongo
import pandas as pd

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

# Step 1 - Scrape Mission to Mars Data

#### Scrape NASA Mars News

In [3]:
# Look at news page for Mars from NASA
url = "https://mars.nasa.gov/news/"
browser.visit(url)

html = browser.html
# Create a Beautiful Soup object
soup = bs(html, 'html.parser')

In [4]:
# Collect latest news title
news_title = soup.find_all('div', class_="content_title")
print(news_title[0].text)

Mars InSight Lander's 'Mole' Pauses Digging


In [7]:
# Collect latest new paragraph
news_p = soup.select('ul.item_list li.slide div.article_teaser_body')
print(news_p[0].text)

After the mission's heat probe began hammering last week, it appears to have hit a rock. The team is analyzing data before they hammer again.


#### Scrape JPL Mars Space Images - Featured Image

In [10]:
# URL of Mars Images website
img_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(img_url)

img_html = browser.html
# Create a Beautiful Soup object 
img_soup = bs(img_html, 'html.parser')

# Collect current featured Mars image
first_img = img_soup.select_one('div.image_and_description_container div.img img.thumb')
first_img_src = first_img['src']

base_img_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize'
featured_img_url = base_img_url + first_img_src

# Print compiled url
print(featured_img_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/spaceimages/images/wallpaper/PIA23070-640x350.jpg


#### Scrape Latest Mars Weather Tweet

In [11]:
# URL of Mars Weather twitter page
twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twitter_url)

twitter_html = browser.html

# Create a Beautiful Soup object
tweet_soup = bs(twitter_html, 'html.parser')

# Save text from mars weather latest tweet
mars_weather_tweet = tweet_soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text
print(mars_weather_tweet)

InSight sol 99 (2019-03-07) low -95.8ºC (-140.4ºF) high -13.3ºC (8.1ºF)
winds from the SW at 4.2 m/s (9.5 mph) gusting to 12.0 m/s (26.8 mph)
pressure at 7.20 hPapic.twitter.com/XRi0faFSv5


#### Scrape Mars Facts

In [13]:
# URL of Space Facts - Mars Facts
facts_url = "https://space-facts.com/mars/"

# Read tables from Space Facts with pandas
facts_table = pd.read_html(facts_url)
facts_table

# Create DataFrame from first table found
facts_df = facts_table[0]

# Convert facts_df to html string
facts_html_table = facts_df.to_html()

# Get rid of any extra newlines to clean up table
facts_html_table = facts_html_table.replace('\n', '')
facts_html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>0</th>      <th>1</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    <

#### Scrape USGS Astrogeology site for images of Mar's hemispheres

In [14]:
# USGS Astrogeology website
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)

hemisphere_html = browser.html

# Create Beautiful Soup object
hemisphere_soup = bs(hemisphere_html, 'html.parser')

In [15]:
# Create list to save titles/captions of mars hemispheres
hemisphere_captions = hemisphere_soup.select("a.itemLink  h3")
captions = []
for attribute in hemisphere_captions:
    caption = attribute.text
    captions.append(caption)
    
captions

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [16]:
# Create list to save high def images of mars hemispheres
hemisphere_urls = hemisphere_soup.select('div.item div.description a')
urls = []

# Perform for loop to iterrate and click through each link to the 
# high def image link and save link
for x in hemisphere_urls:
    url = x['href']
    
#   Create new url to access high def image
    enhanced_url = 'https://astrogeology.usgs.gov' + url
    browser.visit(enhanced_url)
    
    enhanced_html = browser.html
    enhanced_soup = bs(enhanced_html, 'html.parser')
    
#   Use enhanced_soup object to find link to high def image
    almost_img = enhanced_soup.select_one('div.downloads ul li a')
    enhanced_img = almost_img['href']
    
#   Add full enhanced_img_url to the urls list
    urls.append(enhanced_img)
    
# Check urls list
urls

['http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']

In [17]:
# Create list of dictionaries for each hemisphere caption and url
hemisphere_1 = {'title': captions[0], 'img_urls': urls[0]}
hemisphere_2 = {'title': captions[1], 'img_urls': urls[1]}
hemisphere_3 = {'title': captions[2], 'img_urls': urls[2]}
hemisphere_4 = {'title': captions[3], 'img_urls': urls[3]}

# Put hemisphere dictionaries into a list
hemisphere_image_urls = [hemisphere_1, hemisphere_2,
                         hemisphere_3, hemisphere_4]

# Check list of dictionaries
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_urls': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_urls': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_urls': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_urls': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]